#Bài tập thực hành - Lập trình máy học cho python - CS116.M11



MSSV: 19521299

Tên: Nguyễn Chí Cường

Ngày: 2/11/2021

Tuần 9

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Xử lý dữ liệu

In [ ]:
#Bước 1: Đọc data
df = pd.read_csv('Churn_Modelling.csv').drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
#Bước 2: One hot encoding
from sklearn.preprocessing import LabelEncoder, StandardScaler
le_geo = LabelEncoder()
le_gender = LabelEncoder()

df_feat = df.drop('Exited', axis=1)
class_ = df['Exited']

le_geo.fit(df_feat['Geography']) #Dùng trong Bước 7
le_gender.fit(df_feat['Gender']) #Dùng trong bước 7
df_feat['Geography'], df_feat['Gender'] = le_geo.transform(df_feat['Geography']), le_gender.transform(df_feat['Gender'])

In [ ]:
#Bước 3: split data train-test:8-2
X_train, X_test, y_train, y_test = train_test_split(df_feat, class_, test_size=0.2, random_state=42)

In [ ]:
#Bước 4:Scale Data
scale = StandardScaler().fit(X_train) ##Dùng trong Bước 7
X_train_scale = scale.transform(X_train)

#Build Model Neural Network

In [ ]:
#Bước 5: build Model
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(6, input_dim=X_train.shape[-1], activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 66        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 115
Trainable params: 115
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_scale, y_train, epochs=100, batch_size=32)

Epoch 1/100
250/250 [==============================] - 4s 3ms/step - loss: 0.5194 - accuracy: 0.8036
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4660 - accuracy: 0.8116
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4459 - accuracy: 0.8190
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4354 - accuracy: 0.8253
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4286 - accuracy: 0.8285
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4231 - accuracy: 0.8317
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4191 - accuracy: 0.8325
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4158 - accuracy: 0.8340
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4132 - accuracy: 0.8345
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4104 - accura

In [ ]:
#Bước 6: Đánh giá model
X_test_scale = scale.transform(X_test)
_, acc = model.evaluate(X_test_scale, y_test)
accuracy = acc*100
print(f'Accuracy: {accuracy}%')

63/63 [==============================] - 0s 2ms/step - loss: 0.3439 - accuracy: 0.8655
Accuracy: 86.54999732971191%


#Dự đoán kết quả

In [ ]:
#Bước 7: Thử nghiệm xem với khách hàng có thông tin sau thì có nên exit?
def predict_exited(infor):
  #CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary
  infor = {'CreditScore':[infor[0]], 'Geography':[infor[1]], 'Gender':[infor[2]],
         'Age':[infor[3]], 'Tenure':[infor[4]], 'Balance':[infor[5]], 'NumOfProducts':[infor[6]],
         'HasCrCard':[infor[7]], 'IsActiveMember':[infor[8]], 'EstimatedSalary':[infor[9]]}
  data = pd.DataFrame(infor)
  data['Geography'], data['Gender'] = le_geo.transform(data['Geography']), le_gender.transform(data['Gender'])
  feat_scale = scale.transform(data)
  return (model.predict(feat_scale) > 0.5).astype(int)[0][0]

In [ ]:
infor = [600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]
exited = predict_exited(infor)
if exited:
  print('Khách hàng nên exit')
else:
  print('Khách hàng không nên exit')

Khách hàng không nên exit


#Sử dụng K-Flod để đánh giá model

##Xử lý data và model

In [ ]:
#Bước 8: Đánh giá độ chính xác của kiến trúc này bằng phương pháp KFold Cross Validation với K=5
from sklearn.model_selection import KFold

In [ ]:
# build model
def build_model():

  _model = Sequential()
  _model.add(Dense(6, input_dim=X_train.shape[-1], activation='relu'))
  _model.add(Dense(6, activation='relu'))
  _model.add(Dense(1, activation='sigmoid'))
  _model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return _model #model + scale

In [ ]:
def get_accuracy(model, test):
  _, accuracy = model.evaluate(test[0], test[1])
  return accuracy

##Chạy K-Flod

In [21]:
def KFlod_Score(feature, class_):
  score = []
  kf_data = KFold(n_splits=5, random_state=None, shuffle=False).split(feature)
  #Build Model
  model_for_KFold = build_model()
  for train_index, test_index in kf_data:
    #Get feature and class
    feature_train, feature_test = feature[train_index], feature[test_index]
    class_train, class_test = class_[train_index], class_[test_index]
    #fit model
    model_for_KFold.fit(feature_train, class_train, epochs=100, batch_size=32)
    #get accuracy
    score.append(get_accuracy(model_for_KFold, [feature_test, class_test]))

    print("Đã train xong Flod________________________________________________________")
  return score

In [22]:
feature = scale.transform(df_feat)
score = KFlod_Score(feature, class_)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6483 - accuracy: 0.6718
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5122 - accuracy: 0.7974
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4652 - accuracy: 0.7985
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4385 - accuracy: 0.8081
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4260 - accuracy: 0.8161
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4180 - accuracy: 0.8189
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4112 - accuracy: 0.8261
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4039 - accuracy: 0.8295
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3961 - accuracy: 0.8342
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3880 - accura

In [24]:
def average_score(score):
  n = len(score)
  return sum(score)/n

acc_mean = average_score(score)
accuracy = acc_mean * 100
print(f"Accuray trung bình {accuracy} %")

Accuray trung bình 86.1900007724762 %
